In [1]:
import csv
import nltk
import pandas as pd

In [2]:
%run from_columns.ipynb

Reading mt-system.pred with 9280 lines...
Reading smatch-prec with 9280 lines...
Reading trg with 9280 lines...
Reading reference with 9280 lines...
Reading smatch-rec with 9280 lines...
Reading reference.pred with 9280 lines...
Reading src with 9280 lines...
Reading source with 9280 lines...
Reading set with 9280 lines...
Reading mt-system with 9280 lines...
Reading human with 9280 lines...
Reading smatch-f1 with 9280 lines...


In [3]:
# TODO need to make framework for 
# simple smatch
for name in ['smatch-f1', 'smatch-prec', 'smatch-rec']:
    df[name] = [ None if i == 'None\n' else float(i) for i in open('data/trg-en/%s' % name) ]

In [4]:
test = df[df.set.str.contains('newstest2017')]
test_en = test[test.trg == 'en']
test_en[["human"]] = test_en[["human"]].apply(pd.to_numeric)

In [5]:
smoothing_function = nltk.translate.bleu_score.SmoothingFunction()
def sentBLEU(x, y):
    return nltk.translate.bleu_score.sentence_bleu([x], y, smoothing_function=smoothing_function.method1)

In [6]:
# metrics
metrics = [ ('sentence_bleu', sentBLEU) ]
for name, metric in metrics:
    scores = []
    for ref, out in zip(test_en['reference'], test_en['mt-system']):
        scores.append(metric(ref.split(' '), out.split(' ')))
    test_en[name] = scores

In [7]:
# should be 560 for each language
test_en['src'].value_counts()

tr    560
cs    560
ru    560
zh    560
fi    560
lv    560
de    560
Name: src, dtype: int64

In [12]:
print(len(test_en))
test_en['smatch-f1'].describe()

3920


count    3774.000000
mean        0.454080
std         0.190008
min         0.000000
25%         0.333333
50%         0.444444
75%         0.567901
max         1.000000
Name: smatch-f1, dtype: float64

In [8]:
test_en.groupby('src').corr()

human  sentence_bleu  smatch-f1  smatch-prec  smatch-rec
src                                                                           
cs  human          1.000000       0.410560   0.421298     0.386117    0.414957
    sentence_bleu  0.410560       1.000000   0.456158     0.447982    0.427373
    smatch-f1      0.421298       0.456158   1.000000     0.942423    0.933915
    smatch-prec    0.386117       0.447982   0.942423     1.000000    0.784012
    smatch-rec     0.414957       0.427373   0.933915     0.784012    1.000000
de  human          1.000000       0.425359   0.303862     0.311857    0.264624
    sentence_bleu  0.425359       1.000000   0.408922     0.402143    0.373144
    smatch-f1      0.303862       0.408922   1.000000     0.937704    0.937263
    smatch-prec    0.311857       0.402143   0.937704     1.000000    0.772774
    smatch-rec     0.264624       0.373144   0.937263     0.772774    1.000000
fi  human          1.000000       0.560087   0.448977     0.415574    0.436256
    sentence_bleu  0.560087       1.000000   0.593814     0.535152    0.577268
    smatch-f1      0.448977       0.593814   1.000000     0.907196    0.953356
    smatch-prec    0.415574       0.535152   0.907196     1.000000    0.772615
    smatch-rec     0.436256       0.577268   0.953356     0.772615    1.000000
lv  human          1.000000       0.382866   0.327430     0.244332    0.340583
    sentence_bleu  0.382866       1.000000   0.434483     0.374560    0.436315
    smatch-f1      0.327430       0.434483   1.000000     0.894988    0.954369
    smatch-prec    0.244332       0.374560   0.894988     1.000000    0.758244
    smatch-rec     0.340583       0.436315   0.954369     0.758244    1.000000
ru  human          1.000000       0.472228   0.389550     0.347389    0.389964
    sentence_bleu  0.472228       1.000000   0.482280     0.460677    0.456986
    smatch-f1      0.389550       0.482280   1.000000     0.910139    0.920645
    smatch-prec    0.347389       0.460677   0.910139     1.000000    0.742223
    smatch-rec     0.389964       0.456986   0.920645     0.742223    1.000000
tr  human          1.000000       0.521461   0.446487     0.411381    0.423395
    sentence_bleu  0.521461       1.000000   0.510752     0.459789    0.492456
    smatch-f1      0.446487       0.510752   1.000000     0.933904    0.944993
    smatch-prec    0.411381       0.459789   0.933904     1.000000    0.781250
    smatch-rec     0.423395       0.492456   0.944993     0.781250    1.000000
zh  human          1.000000       0.499658   0.458522     0.395374    0.449466
    sentence_bleu  0.499658       1.000000   0.479408     0.444064    0.447188
    smatch-f1      0.458522       0.479408   1.000000     0.925536    0.927352
    smatch-prec    0.395374       0.444064   0.925536     1.000000    0.738372
    smatch-rec     0.449466       0.447188   0.927352     0.738372    1.000000